In [1]:
from pynwb import NWBFile
from datetime import datetime

/Users/xiaoxuanj/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import h5py as h5
import numpy as np

In [3]:
nwb1_file = '/Users/xiaoxuanj/work/work_allen/Ephys/mouse366119/mouse366119_probe_surf.nwb'
nwb1 = h5.File(nwb1_file)

In [4]:
nwb1

<HDF5 file "mouse366119_probe_surf.nwb" (mode r+)>

In [5]:
nwb1['general']['experimenter'].value

b'experimenter'

In [6]:
nwb1['general']['subject']['subject_id'].value

b'subject_id'

In [7]:
nwb1['general']['experiment_description'].value.astype(str)

'with visual stimuli; awake; sorted; unhealthy brain; tail touching grounding; probe C has sync recorded'

In [8]:
str(nwb1['identifier'].value)

"b'366119; NWB-1.0.5; Wed Mar 21 15:17:53 2018'"

In [9]:
nwb2 = NWBFile('NWB 1.0 > NWB 2.0 converter',        # source
                  str(nwb1['session_description'].value), # session description
                  str(nwb1['identifier'].value),          # identifier
                  str(nwb1['session_start_time'].value.astype(str)),  # date
                  experimenter = nwb1['general']['experimenter'].value.astype(str),
                  lab = nwb1['general']['lab'].value.astype(str),
                  institution = nwb1['general']['lab'].value.astype(str),
                  experiment_description = nwb1['general']['experiment_description'].value.astype(str),
                  session_id='SESSION_ID')

In [10]:
device = nwb2.create_device(name='Neuropixels Prototype Rig', source="a source")

In [11]:
probes = [p for p in nwb1['processing'].keys() if p[:5] == 'probe']

In [12]:
probes

['probeA', 'probeB', 'probeC', 'probeD', 'probeE', 'probeF']

In [13]:
locations = {'probeA' : 'AM',
             'probeB' : 'PM',
             'probeC' : 'V1',
             'probeD' : 'LM',
             'probeE' : 'AL',
             'probeF' : 'RL'}

xlocs = [16, 48, 0, 32]

In [14]:
for probe in probes:

    electrode_name = probe
    source = "Neuropixels Probe"
    description = "Phase 3a"
    location = locations[probe]

    electrode_group = nwb2.create_electrode_group(electrode_name,
                                                 source=source,
                                                 description=description,
                                                 location=location,
                                                 device=device)
    
    for idx in range(0,384):
        nwb2.add_electrode(idx,
                          x=float(xlocs[idx % 4]), y=float(idx/2*20), z=np.nan,
                          imp=np.nan,
                          location='unknown', filtering='300 Hz highpass',
                          description='channel %s' % idx, group=electrode_group)

In [15]:
from pynwb.ecephys import Clustering

module = nwb2.create_processing_module('ecephys', # name
                                       'ecephys', #source
                                        'Clustering module' # description
                                      )

for probe in probes:
    
    clustering = Clustering(probe, # source
                                   'Kilosort outputs', # description
                                   nwb1['processing'][probe]['Clustering']['num'].value, # cluster ID
                                   nwb1['processing'][probe]['Clustering']['peak_over_rms'].value, #peak over RMS
                                   nwb1['processing'][probe]['Clustering']['times'].value, # times
                                   probe + '_clustering') # name
    
    module.add_data_interface(clustering)                               


In [19]:
nwb1['processing']['probeA']['Clustering'].keys()

KeysView(<HDF5 group "/processing/probeA/Clustering" (4 members)>)

In [22]:
dir(nwb2.modules['ecephys'].containers[0])

['_Clustering__help',
 '_NWBBaseType__container_source',
 '_NWBBaseType__fields',
 '_NWBBaseType__gather_nwbfields',
 '_NWBBaseType__name',
 '_NWBBaseType__parent',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nwbfields__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_getter',
 '_setter',
 '_to_dict',
 'container_source',
 'description',
 'fields',
 'help',
 'name',
 'namespace',
 'neurodata_type',
 'num',
 'parent',
 'peak_over_rms',
 'source',
 'times',
 'type_hierarchy']

In [23]:
module.containers

(<pynwb.ecephys.Clustering at 0x10baea278>,
 <pynwb.ecephys.Clustering at 0x10baea668>)